In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.utils import plot_model
from keras import backend as K


class_weights = np.array([0,0.5,1,3,3.2,23.5,2.5,163.2,23.5])

def weighted_binary_crossentropy(y_true, y_pred):
       class_loglosses = K.mean(K.binary_crossentropy(y_true, y_pred), axis=[0, 1, 2])
       return K.sum(class_loglosses * K.constant(class_weights))

def Unet(n_classes=9, im_sz=patch_size, n_channels=4,droprate=0.25):
  
    inputs = Input((im_sz, im_sz, n_channels))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(droprate)(pool1)

    
    pool1 = BatchNormalization()(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(droprate)(pool2)

    
    pool2 = BatchNormalization()(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(droprate)(pool3)

    
    conv4 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv4)


    up6 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv4), conv3])
    up6 = BatchNormalization()(up6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = Dropout(droprate)(conv6)

    
    up7 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv6), conv2])
    up7 = BatchNormalization()(up7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = Dropout(droprate)(conv7)

    
    up8 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv7), conv1])
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv8)
    conv9 = Conv2D(9, (1, 1), activation='softmax')(conv8)
    
    model=Model(inputs=[inputs],outputs=[conv9])
    return model 

In [ ]:
# creating generator class to handle large input data easily

from keras.utils import Sequence

class Generator(Sequence):
  
  
  def __init__(self,X,Y,batch):
    self.x=X
    self.y=Y
    self.batch=batch
    self.cnt=0
    
  def __len__(self):
    return int(np.floor(np.array(self.x).shape[0]) / self.batch)
  
  
  def __getitem__(self,index):
    path_x=self.x[index*self.batch:(index+1)*self.batch]                            #list contains paths (number=batch size) of the input patches 
    path_y=self.y[index*(self.batch):(index+1)*self.batch]                          #list contains paths of the corresponding gt patches 
    
    X=[]
    Y=[]
    
    for x_path,y_path in zip(path_x,path_y):
      temp_x=np.load(x_path)                                                        #loading input data from given paths
      temp_y=np.load(y_path)
      #temp_mask=get_mask(temp_x,temp_y)
      self.cnt=self.cnt+1
      X.append(temp_x)            
      Y.append(temp_y) 
    
    X=np.array(X)
    Y=np.array(Y)
      
    return X,Y                                                                       #passing the required list of input and gt patches                                                               
    

In [ ]:
X_patch_paths=["./Image_patch_new/img_patch"+str(i)+".npy" for i in range(0,datasize)]                #storing patch paths to be passed ot the generator
Y_patch_paths=["./mask_patch_new/mask_patch"+str(i)+".npy" for i in range(0,datasize)]

In [ ]:
#splitting data into training and validation set 

X_path_train = X_patch_paths[:train_data]
Y_path_train = Y_patch_paths[:train_data]
X_path_val = X_patch_paths[train_data:]
Y_path_val = Y_patch_paths[train_data:]

In [ ]:
########################## defining the model ##################################

model=Unet(n_classes=9, im_sz=patch_size, n_channels=4,droprate=0.15)


In [ ]:
#model.summary()
from keras.utils import plot_model
plot_model(model, to_file='model.png')
#print(plot_model(model, to_file='model.jpeg'))

In [ ]:
model.compile(optimizer="Adam",loss=weighted_binary_crossentropy, metrics=["accuracy"])

In [ ]:
#defining required generators

training_generator = Generator(X_path_train ,Y_path_train ,6)
val_generator = Generator(X_path_val ,Y_path_val ,6)

In [ ]:
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint("weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', save_best_only=True, verbose=1)

In [ ]:
########################################## Training model over the training data ##########################################


model.fit_generator(generator=training_generator, 
                    use_multiprocessing=True ,
                    validation_data=val_generator,
                    workers=6,
                    callbacks= [model_checkpoint],
                    epochs=20)

print("completed fitting")


In [ ]:
print("saving model")

model.save("./my_model_320_4.h5")


print("model saved")
########################## Training part finished  #############################